# ACAS Xr Coordination

*Implements simple vertical coordination scheme inherited from TCAS/ACAS Xa.*

Also implements coordination scheme for speed (leader/follower terminology):
- Complement: fast-slow, and slow-fast.
- Break ties with address (Mode S, i.e. aircraft index)

Future ideas:
- Faster one, speed up. Slower one, slow down
- What other information can we use to effectively coordinate speeds?


**Make sure you are in a Julia 1.1 env (Robert uses v1.2 but results should be comparable), and that you have aircraftshapes.sty in repo.**

In [103]:
using Revise
includet("src/Xr.jl"); # note: includet for Revise

### Generate simple encounter set

In [104]:
# Generate new encounters from within the Jupyter notebook
@time cd("../UAMEncounterGen") do
    include("generate_nop_nop_encs.jl")
    cp(filename1, joinpath("../XrSim/data_files", basename(filename1)), force=true)
    filename2 === nothing ? nothing : cp(filename2, joinpath("../XrSim/data_files", basename(filename2)), force=true)
end

 17.584190 seconds

┌ Warning: encounter_files/nop_nop_UstarUstar_coord_1.bin already exists... overwriting!
└ @ Main C:\Users\mossr\Code\sisl\ACAS Xr\UAMEncounterGen\generate_nop_nop_encs.jl:15


 (3.63 M allocations: 896.159 MiB, 2.70% gc time)


### 1. Simulating the encounter files

Make sure this notebook is within the UAMEncounterGen repo.

Make sure that your encounter files are in the "data_files" folder within UAMEncounterGenerator. 

In [105]:
# Settings for all aircraft
run_vertical_logic = false
is_responsive = true

encounter_file1 = joinpath("data_files", basename(filename1))
encounter_file2 = filename2 === nothing ? nothing : joinpath("data_files", basename(filename2))

sims = SIMULATION[]

for coordination_enabled = [true, false]
    sim = simulation()

    sim.enc_file = encounter_file1
    for i in 1:2
        if run_vertical_logic
            sim.acs[i] = uam_vert(q_file = "data_files/test.bin")
            sim.acs[i].coordination.enabled = coordination_enabled
        else
            sim.acs[i] = uam_speed(q_file = "data_files/test_speed_120.bin")
            sim.acs[i].perform_scaling = true
            sim.acs[i].coordination.enabled = coordination_enabled
        end
    end

    # turn responsive to false when wanting to check if trajectories are being properly generated
    sim.acs[1].responsive = is_responsive

    xr_sim!(sim);
    
    push!(sims, sim)
end

### Switch ownship and intruder
Helpful when testing leader in coordination scheme (when using tie-breakers by index)

In [106]:
# TODO. Scheme to alter coordination tie-breaking

### Metrics

In [107]:
println(typeof(sims[1].acs[1]))
println("—"^70)
for sim in sims
    println("Coordination: ", sim.acs[1].coordination.enabled)
    println("Alerts: ", sim.sim_out.small_sim_out.alerts, " and NMACS: ", sim.sim_out.small_sim_out.nmacs)
    println("NMAC indices: ", sim.sim_out.small_sim_out.nmac_inds)
    println("—"^50)
end

# println(sim2.sim_out.small_sim_out.alerts)

UAM_SPEED
——————————————————————————————————————————————————————————————————————
Coordination: true
Alerts: 100 and NMACS: 18
NMAC indices: [10, 16, 17, 18, 20, 27, 29, 36, 43, 50, 51, 56, 58, 61, 65, 78, 81, 82]
——————————————————————————————————————————————————
Coordination: false
Alerts: 100 and NMACS: 0
NMAC indices: Int64[]
——————————————————————————————————————————————————


### Single "blip" alerting

In [126]:
time_alerts_held_ac1 = map(actions->sum(actions .!= COC), sims[2].sim_out.ac1_actions) # sims[2] no coordination
time_alerts_held_ac2 = map(actions->sum(actions .!= COC), sims[2].sim_out.ac2_actions) # sims[2] no coordination

println("[ac1] Number of 1 sec 'blip' alerts: ", sum(time_alerts_held_ac1 .== 1), " out of ", length(time_alerts_held_ac1))
println("[ac2] Number of 1 sec 'blip' alerts: ", sum(time_alerts_held_ac2 .== 1), " out of ", length(time_alerts_held_ac2))

[ac1] Number of 1 sec 'blip' alerts: 14 out of 100
[ac2] Number of 1 sec 'blip' alerts: 15 out of 100


### 2. Visualizing the simulated encounter sets

Make sure the simulation you want to visualize does not have sim.sim_out = small_simulation_output()

ac1 = UAM icon

ac2 = quad icon

In [108]:
include("visualization/UAMEncounterViewer.jl");

In [109]:
encounter_viewer(sims[1].sim_out, int_type=:quad, alert_type=run_vertical_logic ? :vert : :speed) # with coord.

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["File Name"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "text",Symbol("data-bind") => "value: value, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}"),:placeholder => "",:className => "input ",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 2 listeners. Value:
0, nothing),"value" => (Observable{String} with 1 listeners. Value:
"myFile.pdf", nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mossr\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mossr\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mossr\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mossr\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mossr\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000003e984b90, Task (runnable) @0x000000003e984b90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"14748993935327084354\",\"id\":\"8404231880744017233\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"14748993935327084354\",\"id\":\"14075162559773212508\",\"type\":\"observable\"})};\n    var self = this;\n    functi

In [110]:
encounter_viewer(sims[2].sim_out, int_type=:quad, alert_type=run_vertical_logic ? :vert : :speed) # without coord.

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["File Name"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "text",Symbol("data-bind") => "value: value, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}"),:placeholder => "",:className => "input ",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 2 listeners. Value:
0, nothing),"value" => (Observable{String} with 1 listeners. Value:
"myFile.pdf", nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\mossr\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\mossr\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\mossr\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\mossr\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\mossr\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000003e46e850, Task (runnable) @0x000000003e46e850), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"13284289846953391483\",\"id\":\"15098860376617306179\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"13284289846953391483\",\"id\":\"16962846095555899908\",\"type\":\"observable\"})};\n    var self = this;\n    funct

In [35]:
# ✈ 🛩